In [1]:
import os
import sys
import getpass

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/Utility Code'.format(user))

import utils_io as utils
import fiona
import arcgis as gis
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
work_dir = os.path.join('/Users',
                        user,
                       'Box',
                       'DataViz Projects',
                       'Spatial Analysis and Mapping',
                       'TOD TOC Policy Update Mapping')

In [3]:
toc_fgdb = os.path.join(work_dir, 
                        'TOD Policy Analysis', 
                        'TOD Policy Analysis.gdb')

## Pre-processing

- Pull General Plan and Zoning Capacity Data (Redshift)
- Pull Parcel Geometries (Socrata)
- Pull TOC Areas (BAM Portal)

In [4]:
pa_parcel_id = '5y7p-4hs4'

In [5]:
# create gdf from toc pda and tra feature class
pda_tra_gdf = gpd.read_file(filename=toc_fgdb, 
                            driver='File GDB', 
                            layer='TOC_PDA_TRA_epsg4326').to_crs('EPSG:26910')

In [6]:
sql = """
SELECT
  parcel_id,
  pa_apn_base as apn,
  county_id as pa_county,
  jurisdiction_id as pa_jurisdiction,
  zn_county,
  zn_jurisdiction,
  zn_recid,
  zn_code,
  zn_description,
  zn_regional_code,
  zn_regional_description,
  zn_min_dua,
  zn_max_dua,
  zn_min_far,
  zn_max_far,
  gp_recid,
  gp_code,
  gp_description,
  gp_regional_code,
  gp_regional_description,
  gp_min_dua,
  gp_max_dua,
  gp_min_far,
  gp_max_far
FROM
  basis_staging.parcel_base_tbl
  LEFT JOIN basis_staging.zn_base_tbl USING (zn_recid)
  LEFT JOIN basis_staging.zn_base_lot_properties USING (zn_recid)
  LEFT JOIN basis_staging.gp_base_tbl Using(gp_recid)
  LEFT JOIN basis_staging.gp_base_density Using (gp_recid)
"""
    
land_use_df = utils.pull_df_from_redshift_sql(sql_statement=sql)

took 1.7878 minutes


In [7]:
# pull parcel atlas parcel data as geojson
pa_parcels_geojson = utils.pull_geojson_from_socrata(socrata_data_id=pa_parcel_id)

pulling 2055059 rows as geojson
took 5.0689 minutes


In [8]:
# convert parcel atlas parcel data to geodataframe
pa_parcels_gdf = gpd.GeoDataFrame.from_features(pa_parcels_geojson, 
                                               crs='EPSG:4326').to_crs('EPSG:26910')

In [9]:
# # join parcels to land use data
# land_use_gdf = pd.merge(pa_parcels_gdf, 
#                         land_use_df, 
#                         how='left', 
#                         on='parcel_id')

## Perform largest area overlay

In [10]:
overlay_fields = [
    'jurisdicti',
    'designatio',
    'area_name',
    'gg_acres',
    'pba2050_transit_investment'
]
parcels_pda_tra_overlay = utils.geo_assign_fields(id_df=pa_parcels_gdf, 
                                                  id_field='parcel_id', 
                                                  overlay_df=pda_tra_gdf, 
                                                  overlay_fields=overlay_fields)

took 7.2504 minutes


In [15]:
overlay_inner = parcels_pda_tra_overlay[parcels_pda_tra_overlay['jurisdicti'].notnull()].copy()

In [16]:
overlay_inner

,parcel_id,jurisdicti,designatio,area_name,gg_acres,pba2050_transit_investment
2069,-121p69584X37p98836X,Oakley,Priority Development Area,Potential Planning Area,232.443156,No Planned Investments
11386,-121p69668X37p94918X,Brentwood,Priority Development Area,Brentwood Blvd,312.158014,No Planned Investments
13453,-121p69732X37p94945X,Brentwood,Priority Development Area,Brentwood Blvd,312.158014,No Planned Investments
13526,-121p69732X37p94918X,Brentwood,Priority Development Area,Brentwood Blvd,312.158014,No Planned Investments
13630,-121p69716X37p94891X,Brentwood,Priority Development Area,Brentwood Blvd,312.158014,No Planned Investments
...,...,...,...,...,...,...
2054810,-122p24052X38p08500X,Vallejo,Priority Development Area,Carquinez Heights PDA,375.147798,Planned Investments
2054817,-122p24991X38p13519X,Vallejo,Priority Development Area,Central Corridor East PDA,936.382867,Planned Investments
2054818,-122p25877X38p11295X,Vallejo,Priority Development Area,Central Corridor West PDA,288.457839,Planned Investments
2054820,-122p25567X38p11460X,Vallejo,Priority Development Area,Central Corridor West PDA,288.457839,Planned Investments


## Merge parcel geometry and land use attrs

In [17]:
pa_parcels_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2055059 entries, 0 to 2055058
Data columns (total 9 columns):
 #   Column        Dtype   
---  ------        -----   
 0   geometry      geometry
 1   jurisdiction  object  
 2   parcel_type   object  
 3   city_soi      object  
 4   x_26910       object  
 5   county        object  
 6   geo_id        object  
 7   parcel_id     object  
 8   y_26910       object  
dtypes: geometry(1), object(8)
memory usage: 141.1+ MB


In [18]:
overlay_gdf = pd.merge(pa_parcels_gdf[['parcel_id','geometry']], 
                       overlay_inner, 
                       how='inner', 
                       on='parcel_id')

In [21]:
lu_overlay_gdf = pd.merge(overlay_gdf, 
                          land_use_df, 
                          how='inner', 
                          on='parcel_id')

## Calculate far and dua capacity

In [37]:
test = np.nan

In [41]:
test2 = -1

In [46]:
if ((pd.isnull(test)) | (test2 <= 0)):
    print('Missing')

In [56]:
def set_capacity_source_col(row, capacity_type):
    if capacity_type == 'residential':
        gp_capacity_col = 'gp_max_dua'
        zn_capacity_col = 'zn_max_dua'
    elif capacity_type == 'commercial':
        gp_capacity_col = 'gp_max_far'
        zn_capacity_col = 'zn_max_far'
    
    if ((pd.isnull(row[zn_capacity_col]) | (row[zn_capacity_col] <= 0)) & 
        (pd.isnull(row[gp_capacity_col]) | (row[gp_capacity_col] <= 0))):
        return 'Missing Capacity'
    elif (pd.isnull(row[zn_capacity_col]) | (row[zn_capacity_col] <= 0)) & (row[gp_capacity_col] > 0):
        return 'General Plan'
    else:
        return 'Zoning'

In [57]:
def calculate_total_capacity(row, capacity_type):
    if capacity_type == 'residential':
        gp_capacity_col = 'gp_max_dua'
        zn_capacity_col = 'zn_max_dua'
        capacity_source = 'res_capacity_source'
    elif capacity_type == 'commercial':
        gp_capacity_col = 'gp_max_far'
        zn_capacity_col = 'zn_max_far'
        capacity_source = 'comm_capacity_source'
        
    if row[capacity_source] == 'Missing Capacity':
        return 0
    elif row[capacity_source] == 'Zoning':
        return (row[zn_capacity_col] * row['acres'])
    else:
        return (row[gp_capacity_col] * row['acres'])

In [58]:
lu_overlay_gdf['parcel_acres'] = (lu_overlay_gdf.geometry.area / 4046.85642)

In [59]:
# set source for residential capacity
lu_overlay_gdf['res_capacity_source'] = lu_overlay_gdf.apply(lambda row: set_capacity_source_col(row, 'residential'), 
                                                             axis=1)

In [60]:
res_inspect_cols = ['jurisdicti','zn_max_dua','gp_max_dua','res_capacity_source']
lu_overlay_gdf[res_inspect_cols][lu_overlay_gdf['jurisdicti'] == 'Alameda']

,jurisdicti,zn_max_dua,gp_max_dua,res_capacity_source
121153,Alameda,16.7,100.0,Zoning
121358,Alameda,NaN,10.0,General Plan
121455,Alameda,9.1,10.0,Zoning
121698,Alameda,9.1,10.0,Zoning
121814,Alameda,9.1,10.0,Zoning
...,...,...,...,...
196842,Alameda,NaN,100.0,General Plan
197119,Alameda,NaN,100.0,General Plan
197144,Alameda,NaN,100.0,General Plan
197149,Alameda,NaN,100.0,General Plan


In [68]:
lu_overlay_gdf[res_inspect_cols][(lu_overlay_gdf['res_capacity_source'] == 'Zoning') &
                                 (lu_overlay_gdf['zn_max_dua'].isnull())]

,jurisdicti,zn_max_dua,gp_max_dua,res_capacity_source


In [69]:
lu_overlay_gdf[res_inspect_cols][(lu_overlay_gdf['res_capacity_source'] == 'General Plan') &
                                 (lu_overlay_gdf['gp_max_dua'].isnull())]

,jurisdicti,zn_max_dua,gp_max_dua,res_capacity_source


In [70]:
lu_overlay_gdf[res_inspect_cols][lu_overlay_gdf['res_capacity_source'] == 'Missing Capacity']

,jurisdicti,zn_max_dua,gp_max_dua,res_capacity_source
0,Oakley,NaN,0.0,Missing Capacity
1,Brentwood,NaN,0.0,Missing Capacity
2,Brentwood,NaN,0.0,Missing Capacity
3,Brentwood,NaN,0.0,Missing Capacity
4,Brentwood,NaN,0.0,Missing Capacity
...,...,...,...,...
507413,Suisun City,NaN,NaN,Missing Capacity
507414,Vallejo,NaN,NaN,Missing Capacity
507418,Vallejo,NaN,NaN,Missing Capacity
507421,Suisun City,NaN,NaN,Missing Capacity


In [71]:
# set source for commercial capacity
lu_overlay_gdf['comm_capacity_source'] = lu_overlay_gdf.apply(lambda row: set_capacity_source_col(row, 'commercial'), 
                                                             axis=1)

In [73]:
comm_inspect_cols = ['jurisdicti','zn_max_far','gp_max_far','comm_capacity_source']
lu_overlay_gdf[comm_inspect_cols][lu_overlay_gdf['jurisdicti'] == 'Alameda']

,jurisdicti,zn_max_far,gp_max_far,comm_capacity_source
121153,Alameda,NaN,NaN,Missing Capacity
121358,Alameda,NaN,1.0,General Plan
121455,Alameda,NaN,1.0,General Plan
121698,Alameda,NaN,1.0,General Plan
121814,Alameda,NaN,1.0,General Plan
...,...,...,...,...
196842,Alameda,NaN,5.0,General Plan
197119,Alameda,NaN,NaN,Missing Capacity
197144,Alameda,NaN,5.0,General Plan
197149,Alameda,NaN,5.0,General Plan
